In [ ]:
import tensorflow as tf
import numpy as np

import sionna as sn

%matplotlib inline
# also try %matplotlib widget

import matplotlib.pyplot as plt

import time

from tensorflow.keras import Model

In [ ]:
NUM_BITS_PER_SYMBOL = 2  # QPSK
constellation = sn.mapping.Constellation("qam", NUM_BITS_PER_SYMBOL)

constellation.show()


In [ ]:
mapper = sn.mapping.Mapper(constellation=constellation)

# The demapper uses the same constellation object as the mapper
demapper = sn.mapping.Demapper("app", constellation=constellation)

binary_source = sn.utils.BinarySource()
awgn_channel = sn.channel.AWGN()

no = sn.utils.ebnodb2no(
    ebno_db=10.0, num_bits_per_symbol=NUM_BITS_PER_SYMBOL, coderate=1.0
)  # Coderate set to 1 as we do uncoded transmission here


In [ ]:
BATCH_SIZE = 64  # How many examples are processed by Sionna in parallel

bits = binary_source([BATCH_SIZE, 1024])  # Blocklength
print("Shape of bits: ", bits.shape)

x = mapper(bits)
print("Shape of x: ", x.shape)

y = awgn_channel([x, no])
print("Shape of y: ", y.shape)

llr = demapper([y, no])
print("Shape of llr: ", llr.shape)


In [ ]:
num_samples = 8  # how many samples shall be printed
num_symbols = int(num_samples / NUM_BITS_PER_SYMBOL)

print(f"First {num_samples} transmitted bits: {bits[0,:num_samples]}")
print(f"First {num_symbols} transmitted symbols: {np.round(x[0,:num_symbols], 2)}")
print(f"First {num_symbols} received symbols: {np.round(y[0,:num_symbols], 2)}")
print(f"First {num_samples} demapped llrs: {np.round(llr[0,:num_samples], 2)}")

plt.figure(figsize=(8, 8))
plt.axes().set_aspect(1)
plt.grid(True)
plt.title("Channel output")
plt.xlabel("Real Part")
plt.ylabel("Imaginary Part")
plt.scatter(tf.math.real(y), tf.math.imag(y))
plt.tight_layout()


In [ ]:
class UncodedSystemAWGN(Model):  # Inherits from Keras Model
    def __init__(self, num_bits_per_symbol, block_length):
        """
        A keras model of an uncoded transmission over the AWGN channel.

        Parameters
        ----------
        num_bits_per_symbol: int
            The number of bits per constellation symbol, e.g., 4 for QAM16.

        block_length: int
            The number of bits per transmitted message block (will be the codeword length later).

        Input
        -----
        batch_size: int
            The batch_size of the Monte-Carlo simulation.

        ebno_db: float
            The `Eb/No` value (=rate-adjusted SNR) in dB.

        Output
        ------
        (bits, llr):
            Tuple:

        bits: tf.float32
            A tensor of shape `[batch_size, block_length] of 0s and 1s
            containing the transmitted information bits.

        llr: tf.float32
            A tensor of shape `[batch_size, block_length] containing the
            received log-likelihood-ratio (LLR) values.
        """

        super().__init__()  # Must call the Keras model initializer

        self.num_bits_per_symbol = num_bits_per_symbol
        self.block_length = block_length
        self.constellation = sn.mapping.Constellation("qam", self.num_bits_per_symbol)
        self.mapper = sn.mapping.Mapper(constellation=self.constellation)
        self.demapper = sn.mapping.Demapper("app", constellation=self.constellation)
        self.binary_source = sn.utils.BinarySource()
        self.awgn_channel = sn.channel.AWGN()

    # @tf.function # Enable graph execution to speed things up
    def __call__(self, batch_size, ebno_db):

        # no channel coding used; we set coderate=1.0
        no = sn.utils.ebnodb2no(
            ebno_db, num_bits_per_symbol=self.num_bits_per_symbol, coderate=1.0
        )

        bits = self.binary_source(
            [batch_size, self.block_length]
        )  # Blocklength set to 1024 bits
        x = self.mapper(bits)
        y = self.awgn_channel([x, no])
        llr = self.demapper([y, no])
        return bits, llr


model_uncoded_awgn = UncodedSystemAWGN(
    num_bits_per_symbol=NUM_BITS_PER_SYMBOL, block_length=1024
)

EBN0_DB_MIN = -3.0  # Minimum value of Eb/N0 [dB] for simulations
EBN0_DB_MAX = 5.0  # Maximum value of Eb/N0 [dB] for simulations
BATCH_SIZE = 2000  # How many examples are processed by Sionna in parallel

ber_plots = sn.utils.PlotBER("AWGN")
ber_plots.simulate(
    model_uncoded_awgn,
    ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 20),
    batch_size=BATCH_SIZE,
    num_target_block_errors=100,  # simulate until 100 block errors occured
    legend="Uncoded",
    soft_estimates=True,
    max_mc_iter=100,  # run 100 Monte-Carlo simulations (each with batch_size samples)
    show_fig=True,
)


In [ ]:
# FEC

k = 12
n = 20

encoder = sn.fec.ldpc.LDPC5GEncoder(k, n)
decoder = sn.fec.ldpc.LDPC5GDecoder(encoder, hard_out=True)

BATCH_SIZE = 1  # one codeword in parallel
u = binary_source([BATCH_SIZE, k])
print("Input bits are: \n", u.numpy())

c = encoder(u)
print("Encoded bits are: \n", c.numpy())


In [ ]:
BATCH_SIZE = 10  # samples per scenario
num_basestations = 4
num_users = 5  # users per basestation
n = 1000  # codeword length per transmitted codeword
coderate = 0.5  # coderate

k = int(coderate * n)  # number of info bits per codeword

# instantiate a new encoder for codewords of length n
encoder = sn.fec.ldpc.LDPC5GEncoder(k, n)

# the decoder must be linked to the encoder (to know the exact code parameters used for encoding)
decoder = sn.fec.ldpc.LDPC5GDecoder(
    encoder,
    hard_out=True,  # binary output or provide soft-estimates
    return_infobits=True,  # or also return (decoded) parity bits
    num_iter=20,  # number of decoding iterations
    cn_type="boxplus-phi",
)  # also try "minsum" decoding

# draw random bits to encode
u = binary_source([BATCH_SIZE, num_basestations, num_users, k])
print("Shape of u: ", u.shape)

# We can immediately encode u for all users, basetation and samples
# This all happens with a single line of code
c = encoder(u)
print("Shape of c: ", c.shape)

print("Total number of processed bits: ", np.prod(c.shape))


In [ ]:
k = 64
n = 128

encoder = sn.fec.polar.Polar5GEncoder(k, n)
decoder = sn.fec.polar.Polar5GDecoder(encoder, dec_type="SCL")  # you can also use "SCL"


In [ ]:
class CodedSystemAWGN(Model):  # Inherits from Keras Model
    def __init__(self, num_bits_per_symbol, n, coderate):
        super().__init__()  # Must call the Keras model initializer

        self.num_bits_per_symbol = num_bits_per_symbol
        self.n = n
        self.k = int(n * coderate)
        self.coderate = coderate
        self.constellation = sn.mapping.Constellation("qam", self.num_bits_per_symbol)

        self.mapper = sn.mapping.Mapper(constellation=self.constellation)
        self.demapper = sn.mapping.Demapper("app", constellation=self.constellation)

        self.binary_source = sn.utils.BinarySource()
        self.awgn_channel = sn.channel.AWGN()

        self.encoder = sn.fec.ldpc.LDPC5GEncoder(self.k, self.n)
        self.decoder = sn.fec.ldpc.LDPC5GDecoder(self.encoder, hard_out=True)

    @tf.function # activate graph execution to speed things up
    def __call__(self, batch_size, ebno_db):
        no = sn.utils.ebnodb2no(
            ebno_db,
            num_bits_per_symbol=self.num_bits_per_symbol,
            coderate=self.coderate,
        )

        bits = self.binary_source([batch_size, self.k])
        codewords = self.encoder(bits)
        x = self.mapper(codewords)
        y = self.awgn_channel([x, no])
        llr = self.demapper([y, no])
        bits_hat = self.decoder(llr)
        return bits, bits_hat


CODERATE = 0.5
BATCH_SIZE = 2000

model_coded_awgn = CodedSystemAWGN(
    num_bits_per_symbol=NUM_BITS_PER_SYMBOL, n=2048, coderate=CODERATE
)

bits, bits_hat = model_coded_awgn(BATCH_SIZE, 5)
print("bits    : ", bits[0][0])
print("bits_hat: ", bits_hat[0][0])

ber_plots.simulate(
    model_coded_awgn,
    ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 15),
    batch_size=BATCH_SIZE,
    num_target_block_errors=100,
    legend="Coded",
    soft_estimates=False,
    max_mc_iter=10,
    show_fig=True,
    forward_keyboard_interrupt=False,
)


In [ ]:
@tf.function()  # enables graph-mode of the following function
def run_graph(batch_size, ebno_db):
    # all code inside this function will be executed in graph mode, also calls of other functions
    print(
        f"Tracing run_graph for values batch_size={batch_size} and ebno_db={ebno_db}."
    )  # print whenever this function is traced
    return model_coded_awgn(batch_size, ebno_db)


In [ ]:
batch_size = 10  # try also different batch sizes
ebno_db = 1.5

# run twice - how does the output change?
run_graph(batch_size, ebno_db)


In [ ]:
# You can print the cached signatures with
print(run_graph.pretty_printed_concrete_signatures())


In [ ]:
repetitions = 4  # average over multiple runs
batch_size = BATCH_SIZE  # try also different batch sizes
ebno_db = 1.5

# --- eager mode ---
t_start = time.perf_counter()
for _ in range(repetitions):
    bits, bits_hat = model_coded_awgn(
        tf.constant(batch_size, tf.int32), tf.constant(ebno_db, tf.float32)
    )
t_stop = time.perf_counter()
# throughput in bit/s
throughput_eager = np.size(bits.numpy()) * repetitions / (t_stop - t_start) / 1e6

print(f"Throughput in Eager mode: {throughput_eager :.3f} Mbit/s")
# --- graph mode ---
# run once to trace graph (ignored for throughput)
run_graph(tf.constant(batch_size, tf.int32), tf.constant(ebno_db, tf.float32))

t_start = time.perf_counter()
for _ in range(repetitions):
    bits, bits_hat = run_graph(
        tf.constant(batch_size, tf.int32), tf.constant(ebno_db, tf.float32)
    )
t_stop = time.perf_counter()
# throughput in bit/s
throughput_graph = np.size(bits.numpy()) * repetitions / (t_stop - t_start) / 1e6

print(f"Throughput in graph mode: {throughput_graph :.3f} Mbit/s")


In [ ]:
ber_plots.simulate(
    run_graph,
    ebno_dbs=np.linspace(EBN0_DB_MIN, EBN0_DB_MAX, 12),
    batch_size=BATCH_SIZE,
    num_target_block_errors=500,
    legend="Coded (Graph mode)",
    soft_estimates=True,
    max_mc_iter=100,
    show_fig=True,
    forward_keyboard_interrupt=False,
)
